Put your screens width and height:

In [ ]:
width, height = 2559, 1599

import dependencies, initialize cascade and video capture

In [ ]:
import numpy as np
import os
import cv2
import pyautogui
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

cascade = cv2.CascadeClassifier("haarcascade_eye.xml")
video_capture = cv2.VideoCapture(0)

define normalize function

In [ ]:
def normalize(x):
    minn, maxx = x.min(), x.max()
    return (x - minn) / (maxx - minn)

define scan function

In [ ]:
def scan(image_size=(117, 62)):
    _, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    boxes = cascade.detectMultiScale(gray, 1.3, 10)
    if len(boxes) == 2:
        eyes = []
        for box in boxes:
            x, y, w, h = box
            eye = frame[y:y + h, x:x + w]
            eye = cv2.resize(eye, image_size)
            eye = normalize(eye)
            eye = eye[10:-10, 5:-5]
            eyes.append(eye)
        return (np.hstack(eyes) * 255).astype(np.uint8)
    else:
        return None

import image data into set X and coordinate data in set Y

In [ ]:
root = './data/'
filepaths = os.listdir(root)
X, Y = [], []
for filepath in filepaths:
    print('loading ' + filepath + ' ...')
    filename = filepath[0:filepath.rfind(".")]
    x, y = filename.split('-')
    x = float(x) / width
    y = float(y) / height
    z = cv2.imread(root + filepath)
    X.append(z)
    Y.append([x, y])
X = np.array(X) / 255.0
Y = np.array(Y)

Build Model

In [ ]:
model = Sequential()
model.add(Conv2D(32, 3, 2, activation = 'relu', input_shape = (62, 117, 3)))
model.add(Conv2D(64, 2, 2, activation = 'relu'))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(2, activation = 'sigmoid'))
model.compile(optimizer = "adam", loss = "mean_squared_error")
model.summary()

epochs = 200
for epoch in range(epochs):
    model.fit(X, Y, batch_size = 32)

print("done")

Scan for eyes and move cursor

In [ ]:
print("scanning starting")

while True:
    eyes = scan()
    print("scanned")
    if not eyes is None:
        print("found eyes")
        eyes = np.expand_dims(eyes / 255.0, axis = 0)
        x, y = model.predict(eyes)[0]
        pyautogui.moveTo(x * width, y * height)